In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="gemma2-9b-it", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000254466B7FE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000254466C49B0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Al Amin and i'm student of North South University ECE department and majoring in CSE.")])

AIMessage(content="Hello Al Amin,\n\nIt's nice to meet you! \n\nThat's great that you're studying Computer Science and Engineering at North South University.  \n\nIs there anything I can help you with today? Perhaps you have a question about a specific programming concept, need help brainstorming an idea for a project, or just want to chat about your studies?  \n\nI'm here to assist in any way I can. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 32, 'total_tokens': 124, 'completion_time': 0.167272727, 'prompt_time': 0.000365258, 'queue_time': 0.012936622, 'total_time': 0.167637985}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-33ec0519-583a-4851-bb99-1c64bf497d2e-0', usage_metadata={'input_tokens': 32, 'output_tokens': 92, 'total_tokens': 124})

In [3]:
from langchain_core.messages import AIMessage
model.invoke(
    [
    HumanMessage(content="Hi, my name is Al Amin and i'm student of North South University ECE department and majoring in CSE."),
    AIMessage(content="Hello Al Amin,\n\nIt's nice to meet you!  North South University's ECE department is well-regarded, and CSE is a fascinating field. \n\nWhat are you most interested in learning about within CSE?  Are you working on any interesting projects right now?\n"),
    HumanMessage(content="What is my name and university name?"),
    ]
)

AIMessage(content='Your name is Al Amin and you are a student at North South University. 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 109, 'total_tokens': 140, 'completion_time': 0.056363636, 'prompt_time': 0.003789433, 'queue_time': 0.009961797, 'total_time': 0.060153069}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-be26900e-4734-40ce-a3b9-91391dc14fdf-0', usage_metadata={'input_tokens': 109, 'output_tokens': 31, 'total_tokens': 140})

## Message History
**We can use a Message History class to wrap our model and make it stateful. This will keep trak of inputs and outputs of the model, and store them in some datstore. Future interactions will then load thosse messages and pass them into the chain as part of the input. Let's see how to use this**

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [5]:
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

In [7]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, my name is Al Amin and i'm student of North South University ECE department and majoring in CSE."),
    ],
    config=config
)
response.content

"Hello Al Amin! \n\nIt's great to meet you.  North South University's ECE department is well-regarded, and CSE is a fascinating field. \n\nWhat are you most enjoying about your studies so far? \n\nDo you have any particular areas of computer science that you're most interested in?\n"

In [8]:
with_message_history.invoke(
    [HumanMessage(content="What is my name and university name and department also?")],
    config=config
)

AIMessage(content="You told me your name is Al Amin, you're a student at North South University, and you're majoring in Computer Science and Engineering (CSE) in the Electrical and Computer Engineering (ECE) department.  \n\nIs there anything else you'd like to tell me about yourself or ask me? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 121, 'total_tokens': 189, 'completion_time': 0.123636364, 'prompt_time': 0.004853339, 'queue_time': 0.008899301, 'total_time': 0.128489703}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8223f484-2dd1-4823-a74a-2f8788f143ff-0', usage_metadata={'input_tokens': 121, 'output_tokens': 68, 'total_tokens': 189})

In [9]:
config2 = {"configurable": {"session_id": "chat2"}}

In [10]:
with_message_history.invoke(
    [HumanMessage(content="Hey, my name is khan")],
    config=config2
)

AIMessage(content="Hello Khan, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.047272727, 'prompt_time': 7.842e-05, 'queue_time': 0.01469109, 'total_time': 0.047351147}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ca15d14-ba50-424f-aece-b61798a3bd5b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41})

In [11]:
with_message_history.invoke(
    [HumanMessage(content="tell me my name!!")],
    config=config2
)

AIMessage(content="You told me your name is Khan!  \n\nIs there something else you'd like to know or talk about? 😄 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 54, 'total_tokens': 83, 'completion_time': 0.052727273, 'prompt_time': 0.001834362, 'queue_time': 0.012928358000000001, 'total_time': 0.054561635}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac9f07fd-16e8-4558-bb57-4e0c24e6079b-0', usage_metadata={'input_tokens': 54, 'output_tokens': 29, 'total_tokens': 83})

# Prompt Template

**Prompt Templates help to trun raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the llm. Let's now make tha a bit more complicated. First, lets add in a system message with some custom instructions (but stilltaking messages as input). Next, we'll add in more input besides just the messages**

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)
chain = prompt|model

In [13]:
chain.invoke({"messages": [HumanMessage(content="Hi, i'm Al Amin")]})

AIMessage(content="Hello Al Amin! Nice to meet you. 👋 \n\nWhat can I do for you today? I'm ready to answer your questions and help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 33, 'total_tokens': 75, 'completion_time': 0.076363636, 'prompt_time': 0.000334549, 'queue_time': 0.014073771, 'total_time': 0.076698185}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3a3de47-9224-4562-9986-fc6c7f5b8483-0', usage_metadata={'input_tokens': 33, 'output_tokens': 42, 'total_tokens': 75})

In [14]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [15]:
config = {"configurable": {"session_id": "chat3"}}

In [16]:
with_message_history.invoke(
    [HumanMessage(content="Hi, i'm Al Amin.")],
    config=config
)

AIMessage(content="Hello Al Amin, it's nice to meet you!  \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 34, 'total_tokens': 61, 'completion_time': 0.049090909, 'prompt_time': 0.000333798, 'queue_time': 0.013206212, 'total_time': 0.049424707}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5059f8c4-c4c5-45d1-8570-aafc64e1ed83-0', usage_metadata={'input_tokens': 34, 'output_tokens': 27, 'total_tokens': 61})

In [17]:
## add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model


In [18]:
response = chain.invoke({"messages": [HumanMessage(content="Hi my name is Al Amin")], "language": "Urdu"})
response.content

"Salam Al Amin!  \n\nPleased to meet you. 👋 \n\nWhat can I do for you today? \n\nI'm here to help with any questions you have, translate things, write different kinds of creative content, and more. Just ask! 😊 \n\n"

In [19]:
## add more complexity
with_message_history=RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat4"}}

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, i'm alamin and student of North South University and ECE department.")], "language": "Urdu"},
    config=config
)

response.content

"Assalam o Alaikum Alamin! \n\nIt's nice to meet you. I'm glad you reached out.  \n\nAs a helpful assistant, I'll do my best to answer your questions. What can I help you with today, related to your studies at North South University's ECE department or anything else?  \n\nI can try to:\n\n* **Provide information about ECE concepts:** If you have questions about specific topics in your coursework, I might be able to offer some explanations or insights.\n* **Help with research:** Need to find information about a particular technology or research paper? I can help you search for relevant resources.\n* **Brainstorm ideas:** Stuck on a project? I can help you generate some creative ideas.\n* **Practice your Urdu:**  If you want to improve your Urdu, we can chat in Urdu!\n\n\nJust let me know how I can be of service. 😊\n\n"

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="what is my name?")], "language": "Urdu"},
    config=config
)

response.content

'Your name is Alamin.  😊 \n\nI remember!  You told me at the beginning of our conversation.  \n\n\n\n'

## Managing the conversation History

One important concept to understand when building chatbots is how to manage conversation history. if left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
**trim_messages** The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system messages and wether to allow partial messages

In [26]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    token_counter = model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="Hi, i'm Al Amin"),
    AIMessage(content="Hi, Al Amin nice to meet you!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content='Nice'),
    HumanMessage(content="What is 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content='Thanks'),
    AIMessage(content="mention not."),
    HumanMessage(content="having fun!!"),
    AIMessage(content='Yes!!')
]

In [27]:
trimmer.invoke(messages)

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mod

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='mention not.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun!!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!!', additional_kwargs={}, response_metadata={})]

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like?")],
        "language": "English"
    }
)

AIMessage(content="As an AI, I don't have access to your personal information, including your favorite ice cream flavor!  \n\nWhat's your favorite ice cream?  🍨🍦\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 85, 'total_tokens': 123, 'completion_time': 0.069090909, 'prompt_time': 0.002381471, 'queue_time': 0.012158367, 'total_time': 0.07147238}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e1067827-17bb-47fb-af54-a7525a2ac5c6-0', usage_metadata={'input_tokens': 85, 'output_tokens': 38, 'total_tokens': 123})

In [29]:
chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English"
    }
)

AIMessage(content='You asked "What is 2 + 2?"  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 85, 'total_tokens': 100, 'completion_time': 0.027272727, 'prompt_time': 0.003229757, 'queue_time': 0.011498811, 'total_time': 0.030502484}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-713b663f-2eb7-4e42-b5c9-b2c21eddf075-0', usage_metadata={'input_tokens': 85, 'output_tokens': 15, 'total_tokens': 100})

In [30]:
## LEts wrap this in the message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable": {"session_id": "chat5"}}

In [32]:
chain.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English"
    },
    config=config,
)

AIMessage(content="As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just type it out and I'll do my best to solve it.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 32, 'total_tokens': 87, 'completion_time': 0.1, 'prompt_time': 0.001222215, 'queue_time': 0.013999652, 'total_time': 0.101222215}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6cd2f5e6-4b93-431f-a31a-8e109c2c9f37-0', usage_metadata={'input_tokens': 32, 'output_tokens': 55, 'total_tokens': 87})

In [33]:
chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English"
    },
    config=config,
)

AIMessage(content='You asked "What is 2 + 2?" 😊  \n\n\n\nLet me know if you\'d like to try another one!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 85, 'total_tokens': 115, 'completion_time': 0.054545455, 'prompt_time': 0.003255758, 'queue_time': 0.011141721, 'total_time': 0.057801213}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-94570c77-72b6-4463-b201-7a06dc8f7470-0', usage_metadata={'input_tokens': 85, 'output_tokens': 30, 'total_tokens': 115})